In [63]:
# Import packages

import pandas as pd 
import numpy as np 
import os
from bs4 import BeautifulSoup
import time
import requests
import nfl_data_py as nfl
import matplotlib.pyplot as plt
import plotly
import plotly.express as px

# Set certain settings for the notebook
pd.set_option("display.max_columns", None)

In [64]:
# Check current directory
os.getcwd()

'C:\\Users\\imacd_0odruq3\\Documents\\sports_gambling\\sports_gambling_algorithm'

In [65]:
# Use nfl_data_py package to get data
# Only need data from the current season and possibly previous season for predictions

current_season = 2022
previous_season = current_season-1

nfl_df = nfl.import_pbp_data([previous_season, current_season])

2021 done.
Data not available for 2022
Downcasting floats.


In [66]:
# Add column detailing if there was a qb designed run

rosters = nfl.import_rosters([previous_season, current_season])
qb_roster = rosters[rosters.position == 'QB']
qb_roster = qb_roster[['position', 'player_name', 'player_id']].drop_duplicates()
qb_roster['player_id_string'] = qb_roster.apply(lambda x: str(x.player_id), axis=1)

def is_designed_qb_run(play, qb_roster):
    if play.rusher_id:
        if qb_roster.player_id_string.str.contains(str(play.rusher_id)).any():
            return 1 
    else: 
        return 0 
    
nfl_df['qb_designed_run'] = nfl_df.apply(lambda x: is_designed_qb_run(x, qb_roster), axis=1)

In [67]:
# Get important offensive NFL stats per game per team

# Get only offensive plays, take out qb kneels

offense = nfl_df[nfl_df.play_type.isin(['run', 'pass'])]
offense = offense[offense.qb_kneel == 0]
# offense

In [68]:
# Get rushing stats

# Total rushes, total rush yards, rushing epa

rushes = offense[offense.play_type == 'run']
rushing_grouped = rushes.groupby(by=['season', 'week', 'posteam'])
rush_df = rushing_grouped.count()['play_id'].rename('total_rushes').to_frame()
rush_df[['total_rush_yards', 'rushing_epa', 'rush_tds']] = rushing_grouped.sum()[['yards_gained', 'epa', 'rush_touchdown']]

# rush_df

In [69]:
# Get passing stats

# Total pass attempts, total pass yards, passing epa

passing = offense[offense.play_type == 'pass']
passing_grouped = passing.groupby(by=['season', 'week', 'posteam'])
passing_grouped_no_sacks = passing[passing.sack == 0].groupby(by=['season', 'week', 'posteam'])
pass_df = passing_grouped_no_sacks.count()['play_id'].rename('total_pass_attempts').to_frame()
pass_df['total_passing_yards'] = passing_grouped_no_sacks.sum()['yards_gained']
pass_df[['completions', 'passing_epa', 'pass_tds']] = passing_grouped.sum()[['complete_pass', 'epa', 'pass_touchdown']]
pass_df['avg_cpoe'] = passing_grouped_no_sacks.mean()['cpoe'].rename('avg_cpoe').to_frame()

# pass_df

In [70]:
# Get other offensive stats

# Get qb epa 
qb_stats = offense[(offense.play_type == 'pass') | (offense.qb_scramble == 1) | (offense.qb_designed_run == 1)]
qb_epa_df = qb_stats.groupby(by = ['season', 'week', 'posteam']).sum()['epa'].rename('qb_epa').to_frame()

# Get sacks, interceptions, fumbles, fumbles lost, turnovers
turnovers_df = offense.groupby(by = ['season', 'week', 'posteam']).sum()[['sack', 'fumble', 'fumble_lost', 'interception']]
turnovers_df.columns = ['sacks_allowed', 'fumbles', 'lost_fumbles', 'interceptions_thrown']
# turnovers_df

In [71]:
# Get important defensive NFL stats per game per team

# Get only defensive plays, take out qb kneels

defense = nfl_df[nfl_df.play_type.isin(['run', 'pass'])]
defense = defense[defense.qb_kneel == 0]
defense

,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,side_of_field,yardline_100,game_date,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,game_half,quarter_end,drive,sp,qtr,down,goal_to_go,time,yrdln,ydstogo,ydsnet,desc,play_type,yards_gained,shotgun,no_huddle,qb_dropback,qb_kneel,qb_spike,qb_scramble,pass_length,pass_location,air_yards,yards_after_catch,run_location,run_gap,field_goal_result,kick_distance,extra_point_result,two_point_conv_result,home_timeouts_remaining,away_timeouts_remaining,timeout,timeout_team,td_team,td_player_name,td_player_id,posteam_timeouts_remaining,defteam_timeouts_remaining,total_home_score,total_away_score,posteam_score,defteam_score,score_differential,posteam_score_post,defteam_score_post,score_differential_post,no_score_prob,opp_fg_prob,opp_safety_prob,opp_td_prob,fg_prob,safety_prob,td_prob,extra_point_prob,two_point_conversion_prob,ep,epa,total_home_epa,total_away_epa,total_home_rush_epa,total_away_rush_epa,total_home_pass_epa,total_away_pass_epa,air_epa,yac_epa,comp_air_epa,comp_yac_epa,total_home_comp_air_epa,total_away_comp_air_epa,total_home_comp_yac_epa,total_away_comp_yac_epa,total_home_raw_air_epa,total_away_raw_air_epa,total_home_raw_yac_epa,total_away_raw_yac_epa,wp,def_wp,home_wp,away_wp,wpa,vegas_wpa,vegas_home_wpa,home_wp_post,away_wp_post,vegas_wp,vegas_home_wp,total_home_rush_wpa,total_away_rush_wpa,total_home_pass_wpa,total_away_pass_wpa,air_wpa,yac_wpa,comp_air_wpa,comp_yac_wpa,total_home_comp_air_wpa,total_away_comp_air_wpa,total_home_comp_yac_wpa,total_away_comp_yac_wpa,total_home_raw_air_wpa,total_away_raw_air_wpa,total_home_raw_yac_wpa,total_away_raw_yac_wpa,punt_blocked,first_down_rush,first_down_pass,first_down_penalty,third_down_converted,third_down_failed,fourth_down_converted,fourth_down_failed,incomplete_pass,touchback,interception,punt_inside_twenty,punt_in_endzone,punt_out_of_bounds,punt_downed,punt_fair_catch,kickoff_inside_twenty,kickoff_in_endzone,kickoff_out_of_bounds,kickoff_downed,kickoff_fair_catch,fumble_forced,fumble_not_forced,fumble_out_of_bounds,solo_tackle,safety,penalty,tackled_for_loss,fumble_lost,own_kickoff_recovery,own_kickoff_recovery_td,qb_hit,rush_attempt,pass_attempt,sack,touchdown,pass_touchdown,rush_touchdown,return_touchdown,extra_point_attempt,two_point_attempt,field_goal_attempt,kickoff_attempt,punt_attempt,fumble,complete_pass,assist_tackle,lateral_reception,lateral_rush,lateral_return,lateral_recovery,passer_player_id,passer_player_name,passing_yards,receiver_player_id,receiver_player_name,receiving_yards,rusher_player_id,rusher_player_name,rushing_yards,lateral_receiver_player_id,lateral_receiver_player_name,lateral_receiving_yards,lateral_rusher_player_id,lateral_rusher_player_name,lateral_rushing_yards,lateral_sack_player_id,lateral_sack_player_name,interception_player_id,interception_player_name,lateral_interception_player_id,lateral_interception_player_name,punt_returner_player_id,punt_returner_player_name,lateral_punt_returner_player_id,lateral_punt_returner_player_name,kickoff_returner_player_name,kickoff_returner_player_id,lateral_kickoff_returner_player_id,lateral_kickoff_returner_player_name,punter_player_id,punter_player_name,kicker_player_name,kicker_player_id,own_kickoff_recovery_player_id,own_kickoff_recovery_player_name,blocked_player_id,blocked_player_name,tackle_for_loss_1_player_id,tackle_for_loss_1_player_name,tackle_for_loss_2_player_id,tackle_for_loss_2_player_name,qb_hit_1_player_id,qb_hit_1_player_name,qb_hit_2_player_id,qb_hit_2_player_name,forced_fumble_player_1_team,forced_fumble_player_1_player_id,forced_fumble_player_1_player_name,forced_fumble_player_2_team,forced_fumble_player_2_player_id,forced_fumble_player_2_player_name,solo_tackle_1_team,solo_tackle_2_team,solo_tackle_1_player_id,solo_tackle_2_player_id,solo_tackle_1_player_name,solo_tackle_2_player_name,assist_tackle_1_player_id,assist_tackle_1_player_name,assist_tackle_1_team,assist_tack

In [72]:
# Get rushing defense stats

# Total rushes, total rush yards, rushing epa

rushes_def = defense[defense.play_type == 'run']
rushing_grouped_def = rushes.groupby(by=['season', 'week', 'defteam'])
rush_df_def = rushing_grouped_def.count()['play_id'].rename('total_rushes_allowed').to_frame()
rush_df_def[['total_rush_yards_allowed', 'rushing_epa_allowed', 'rush_tds_allowed']] = rushing_grouped_def.sum()[['yards_gained', 'epa', 'rush_touchdown']]

# rush_df_def

In [73]:
# Get passing defense stats

# Total pass attempts, total pass yards, passing epa

passing_def = defense[defense.play_type == 'pass']
passing_grouped_def = passing_def.groupby(by=['season', 'week', 'defteam'])
passing_grouped_def_no_sacks = passing_def[passing_def.sack == 0].groupby(by=['season', 'week', 'defteam'])
pass_df_def = passing_grouped_def_no_sacks.count()['play_id'].rename('total_pass_attempts_allowed').to_frame()
pass_df_def['passing_yards_allowed'] = passing_grouped_def_no_sacks.sum()['yards_gained']
pass_df_def[['completions_allowed', 'passing_epa_allowed', 'pass_tds_allowed']] = passing_grouped_def.sum()[['complete_pass', 'epa', 'pass_touchdown']]
pass_df_def['avg_cpoe_allowed'] = passing_grouped_def.mean()['cpoe'].rename('avg_cpoe').to_frame()

# pass_df_def

In [74]:
# Get other defensive stats

# Get qb epa 
qb_stats_def = defense[(defense.play_type == 'pass') | (defense.qb_scramble == 1) | (defense.qb_designed_run == 1)]
qb_epa_df_def = qb_stats_def.groupby(by = ['season', 'week', 'defteam']).sum()['epa'].rename('qb_epa_allowed').to_frame()

# Get sacks, interceptions, fumbles, fumbles lost, turnovers
turnovers_df_def = defense.groupby(by = ['season', 'week', 'defteam']).sum()[['sack', 'fumble_forced', 'interception']]
turnovers_df_def.columns = ['sacks', 'fumbles_forced', 'interceptions']
fumble_recovery_df = defense[(defense.defteam == defense.fumble_recovery_1_team)].groupby(by = ['season', 'week', 'defteam']).sum()['fumble_forced'].rename('fumbles_recovered').to_frame()
turnovers_df_def = turnovers_df_def.join(fumble_recovery_df, how='left').fillna(0)

# turnovers_df_def

In [75]:
# Get special teams stats

specials = nfl_df[nfl_df.special == 1]
specials_grouped = specials.groupby(by = ['season', 'week', 'posteam'])
specials_epa_df_one = specials_grouped.sum()['epa'].rename('special_teams_epa_one').to_frame()

specials = nfl_df[nfl_df.special == 1]
specials_grouped = specials.groupby(by = ['season', 'week', 'defteam'])
specials_epa_df_two = specials_grouped.sum()['epa'].rename('special_teams_epa_two').to_frame()

specials_epa_df = specials_epa_df_one.merge(specials_epa_df_two, left_on=['season', 'week', 'posteam'], right_index=True)
specials_epa_df['special_teams_epa'] = specials_epa_df.special_teams_epa_one - specials_epa_df.special_teams_epa_two
specials_epa_df.drop(columns = ['special_teams_epa_one', 'special_teams_epa_two'], inplace=True)

# specials_epa_df

In [76]:
# Get overall score and other game total stats

home_scores_and_etc_df = nfl_df.groupby(by = ['season', 'week', 'home_team', 'away_team']).max()[['home_score', 'away_score']]
home_scores_and_etc_df.index.names = ['season', 'week', 'team', 'opponent']
home_scores_and_etc_df.columns = ['score', 'opponent_score']
away_scores_and_etc_df = nfl_df.groupby(by = ['season', 'week', 'away_team', 'home_team']).max()[['away_score', 'home_score']]
away_scores_and_etc_df.index.names = ['season', 'week', 'team', 'opponent']
away_scores_and_etc_df.columns = ['score', 'opponent_score']

final_nfl_df = pd.concat([home_scores_and_etc_df, away_scores_and_etc_df])
final_nfl_df = final_nfl_df.sort_index()

# final_nfl_df

<ipython-input-76-3ac3bb5220d8>:3: FutureWarning: Dropping invalid columns in DataFrameGroupBy.max is deprecated. In a future version, a TypeError will be raised. Before calling .max, select only columns which should be valid for the function.
  home_scores_and_etc_df = nfl_df.groupby(by = ['season', 'week', 'home_team', 'away_team']).max()[['home_score', 'away_score']]
<ipython-input-76-3ac3bb5220d8>:6: FutureWarning: Dropping invalid columns in DataFrameGroupBy.max is deprecated. In a future version, a TypeError will be raised. Before calling .max, select only columns which should be valid for the function.
  away_scores_and_etc_df = nfl_df.groupby(by = ['season', 'week', 'away_team', 'home_team']).max()[['away_score', 'home_score']]


In [77]:
# Combine all stats together into one data frame

final_nfl_df = final_nfl_df.merge(
    pass_df, left_on=['season', 'week', 'team'], right_index=True).merge(
    rush_df, left_on=['season', 'week', 'team'], right_index=True).merge(
    qb_epa_df, left_on=['season', 'week', 'team'], right_index=True).merge(
    turnovers_df, left_on=['season', 'week', 'team'], right_index=True).merge(
    rush_df_def, left_on=['season', 'week', 'team'], right_index=True).merge(
    pass_df_def, left_on=['season', 'week', 'team'], right_index=True).merge(
    turnovers_df_def, left_on=['season', 'week', 'team'], right_index=True).merge(
    specials_epa_df, left_on=['season', 'week', 'team'], right_index=True)

# final_nfl_df

In [78]:
# Flip sign of def epa 

final_nfl_df['passing_epa_def'] = -final_nfl_df['passing_epa_allowed']
final_nfl_df['rushing_epa_def'] = -final_nfl_df['rushing_epa_allowed']

# Add total epa stats

final_nfl_df['total_epa'] = final_nfl_df.apply(lambda x: x.passing_epa + x.rushing_epa + x.special_teams_epa + x.rushing_epa_def + x.passing_epa_def, axis = 1)
final_nfl_df['total_opposing_epa'] = -final_nfl_df.total_epa

In [79]:
# View final nfl data frame

# final_nfl_df

In [80]:
# Store copy of final_nfl_df, rename it

nfl_per_game = final_nfl_df.copy()

In [81]:
# Read in elo data

elo = pd.read_csv('https://projects.fivethirtyeight.com/nfl-api/nfl_elo.csv')

# elo

,date,season,neutral,playoff,team1,team2,elo1_pre,elo2_pre,elo_prob1,elo_prob2,elo1_post,elo2_post,qbelo1_pre,qbelo2_pre,qb1,qb2,qb1_value_pre,qb2_value_pre,qb1_adj,qb2_adj,qbelo_prob1,qbelo_prob2,qb1_game_value,qb2_game_value,qb1_value_post,qb2_value_post,qbelo1_post,qbelo2_post,score1,score2,quality,importance,total_rating
0,1920-09-26,1920,0,NaN,RII,STP,1503.947000,1300.000000,0.824651,0.175349,1516.108000,1287.838000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48,0,NaN,NaN,NaN
1,1920-10-03,1920,0,NaN,CBD,PTQ,1504.688000,1300.000000,0.825267,0.174733,1516.803000,1287.885000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48,0,NaN,NaN,NaN
2,1920-10-03,1920,0,NaN,CHI,MUT,1368.333000,1300.000000,0.682986,0.317014,1386.533000,1281.800000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20,0,NaN,NaN,NaN
3,1920-10-03,1920,0,NaN,RII,MUN,1516.108000,1478.004000,0.644171,0.355829,1542.135000,1451.977000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45,0,NaN,NaN,NaN
4,1920-10-03,1920,0,NaN,DAY,COL,1493.002000,1504.908000,0.575819,0.424181,1515.434000,1482.475000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17090,2022-01-23,2021,0,d,TB,LAR,1681.797158,1632.341105,0.659005,0.340995,1662.522916,1651.615346,1649.547292,1615.165080,Tom Brady,Matthew Stafford,249.882409,179.870436,11.456236,-0.623739,0.681440,0.318560,132.180809,379.027146,238.112249,199.786107,1629.446888,1635.265484,27,30,96.0,100.0,98.0
17091,2022-01-23,2021,0,d,KC,BUF,1699.470145,1683.332367,0.614689,0.385311,1713.932369,1668.870143,1687.450699,1656.910634,Patrick Mahomes,Josh Allen,273.771767,275.630099,20.577067,18.202487,0.652720,0.347280,622.042187,434.101420,308.598809,291.477231,1700.324756,1644.036577,42,36,100.0,100.0,100.0
17092,2022-01-30,2021,0,c,KC,CIN,1713.932369,1569.498961,0.769519,0.230481,1690.352000,1593.079331,1700.324756,1559.717895,Patrick Mahomes,Joe Burrow,308.598809,190.637925,36.961734,24.405309,0.810417,0.189583,134.683170,153.972640,291.207245,186.971396,1674.943733,1585.098917,24,27,98.0,100.0,99.0
17093,2022-01-30,2021,0,c,LAR,SF,1651.615346,1621.465562,0.633608,0.366392,1661.352751,1611.728157,1635.265484,1631.632019,Matthew Stafford,Jimmy Garoppolo,199.786107,117.398901,9.365284,-11.464001,0.636093,0.363907,248.504596,197.148060,204.657956,125.373817,1644.929010,1621.968494,20,17,96.0,100.0,98.0


In [82]:
# Filter elo to only have data from current and previous season

elo = elo[elo.season >= previous_season]

In [84]:
# Drop some unnecesssary columns, rename some columns to match our other conventions

elo = elo.drop(columns = ['neutral', 'playoff'])
elo = elo.rename(columns = {'team1': 'home', 'team2': 'away', 'score1':'home_score', 'score2':'away_score'})

,date,season,home,away,elo1_pre,elo2_pre,elo_prob1,elo_prob2,elo1_post,elo2_post,qbelo1_pre,qbelo2_pre,qb1,qb2,qb1_value_pre,qb2_value_pre,qb1_adj,qb2_adj,qbelo_prob1,qbelo_prob2,qb1_game_value,qb2_game_value,qb1_value_post,qb2_value_post,qbelo1_post,qbelo2_post,home_score,away_score,quality,importance,total_rating
16810,2021-09-09,2021,TB,DAL,1656.235930,1452.424236,0.824539,0.175461,1659.671437,1448.988729,1634.451744,1505.628824,Tom Brady,Dak Prescott,213.410550,231.491799,8.527869,17.881196,0.736090,0.263910,317.128566,397.401725,223.782352,248.082791,1639.815956,1500.264612,31,29,85.0,48.0,67.0
16811,2021-09-12,2021,IND,SEA,1561.490084,1565.751134,0.586531,0.413469,1530.547371,1596.693848,1527.521731,1553.102962,Carson Wentz,Russell Wilson,102.469223,187.030653,-21.756851,-0.326869,0.522245,0.477755,216.666972,246.652318,113.888998,192.992819,1500.541394,1580.083300,16,28,63.0,73.0,68.0
16812,2021-09-12,2021,TEN,ARI,1553.397217,1486.436956,0.681272,0.318728,1506.171519,1533.662654,1534.793055,1483.219040,Ryan Tannehill,Kyler Murray,207.780411,217.216865,6.311825,5.662860,0.657171,0.342829,33.684565,302.017311,190.370826,225.696909,1489.651053,1528.361042,13,38,56.0,59.0,58.0
16813,2021-09-12,2021,DET,SF,1390.645249,1490.665951,0.449771,0.550229,1371.189983,1510.121217,1369.557634,1548.057844,Jared Goff,Jimmy Garoppolo,160.639094,127.748569,-1.519108,-4.082609,0.343427,0.656573,269.384920,175.216924,171.513676,132.495404,1355.200609,1562.414869,33,41,23.0,45.0,34.0
16814,2021-09-12,2021,ATL,PHI,1484.493617,1426.465972,0.670004,0.329996,1437.713081,1473.246508,1462.273784,1432.000543,Matt Ryan,Jalen Hurts,178.596517,119.529623,0.995672,-7.458145,0.635255,0.364745,17.562049,382.369867,162.493070,145.813648,1418.481268,1475.793058,6,32,21.0,57.0,39.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17090,2022-01-23,2021,TB,LAR,1681.797158,1632.341105,0.659005,0.340995,1662.522916,1651.615346,1649.547292,1615.165080,Tom Brady,Matthew Stafford,249.882409,179.870436,11.456236,-0.623739,0.681440,0.318560,132.180809,379.027146,238.112249,199.786107,1629.446888,1635.265484,27,30,96.0,100.0,98.0
17091,2022-01-23,2021,KC,BUF,1699.470145,1683.332367,0.614689,0.385311,1713.932369,1668.870143,1687.450699,1656.910634,Patrick Mahomes,Josh Allen,273.771767,275.630099,20.577067,18.202487,0.652720,0.347280,622.042187,434.101420,308.598809,291.477231,1700.324756,1644.036577,42,36,100.0,100.0,100.0
17092,2022-01-30,2021,KC,CIN,1713.932369,1569.498961,0.769519,0.230481,1690.352000,1593.079331,1700.324756,1559.717895,Patrick Mahomes,Joe Burrow,308.598809,190.637925,36.961734,24.405309,0.810417,0.189583,134.683170,153.972640,291.207245,186.971396,1674.943733,1585.098917,24,27,98.0,100.0,99.0
17093,2022-01-30,2021,LAR,SF,1651.615346,1621.465562,0.633608,0.366392,1661.352751,1611.728157,1635.265484,1631.632019,Matthew Stafford,Jimmy Garoppolo,199.786107,117.398901,9.365284,-11.464001,0.636093,0.363907,248.504596,197.148060,204.657956,125.373817,1644.929010,1621.968494,20,17,96.0,100.0,98.0


In [85]:
# Map team names from abbreviations to full names
# Note: for any teams that changed their names since the 2014 season, all games will refer to them using their current 
# name, even if they had a different name at the time of the game. 

def fix_home_team_names(game):
    team_mapping = {
        'ARI':'Arizona Cardinals',
        'ATL':'Atlanta Falcons',
        'BAL':'Baltimore Ravens',
        'BUF':'Buffalo Bills',
        'CAR':'Carolina Panthers',
        'CHI':'Chicago Bears',
        'CIN':'Cincinnati Bengals',
        'CLE':'Cleveland Browns',
        'DAL':'Dallas Cowboys',
        'DEN':'Denver Broncos',
        'DET':'Detroit Lions',
        'GB':'Green Bay Packers',
        'HOU':'Houston Texans',
        'IND':'Indianapolis Colts',
        'JAX':'Jacksonville Jaguars',
        'KC':'Kansas City Chiefs',
        'OAK':'Las Vegas Raiders',
        'LAC':'Los Angeles Chargers',
        'LAR':'Los Angeles Rams',
        'MIA':'Miami Dolphins',
        'MIN':'Minnesota Vikings',
        'NE':'New England Patriots',
        'NO':'New Orleans Saints',
        'NYG':'New York Giants',
        'NYJ':'New York Jets',
        'PHI':'Philadelphia Eagles',
        'PIT':'Pittsburgh Steelers',
        'SF':'San Francisco 49ers',
        'SEA':'Seattle Seahawks',
        'TB':'Tampa Bay Buccaneers',
        'TEN':'Tennessee Titans',
        'WSH':'Washington Football Team'
    }
    
    return team_mapping[game['home']]
    
def fix_away_team_names(game):
    team_mapping = {
        'ARI':'Arizona Cardinals',
        'ATL':'Atlanta Falcons',
        'BAL':'Baltimore Ravens',
        'BUF':'Buffalo Bills',
        'CAR':'Carolina Panthers',
        'CHI':'Chicago Bears',
        'CIN':'Cincinnati Bengals',
        'CLE':'Cleveland Browns',
        'DAL':'Dallas Cowboys',
        'DEN':'Denver Broncos',
        'DET':'Detroit Lions',
        'GB':'Green Bay Packers',
        'HOU':'Houston Texans',
        'IND':'Indianapolis Colts',
        'JAX':'Jacksonville Jaguars',
        'KC':'Kansas City Chiefs',
        'OAK':'Las Vegas Raiders',
        'LAC':'Los Angeles Chargers',
        'LAR':'Los Angeles Rams',
        'MIA':'Miami Dolphins',
        'MIN':'Minnesota Vikings',
        'NE':'New England Patriots',
        'NO':'New Orleans Saints',
        'NYG':'New York Giants',
        'NYJ':'New York Jets',
        'PHI':'Philadelphia Eagles',
        'PIT':'Pittsburgh Steelers',
        'SF':'San Francisco 49ers',
        'SEA':'Seattle Seahawks',
        'TB':'Tampa Bay Buccaneers',
        'TEN':'Tennessee Titans',
        'WSH':'Washington Football Team'
    }
    
    return team_mapping[game['away']]


elo['home_full_name'] = elo.apply(lambda x: fix_home_team_names(x), axis=1)
elo['away_full_name'] = elo.apply(lambda x: fix_away_team_names(x), axis=1)

In [87]:
# Store copy of elo, rename it

nfl_elo = elo.copy()

# nfl_elo

In [89]:
# Add full team names to nfl data 

# Map team names from abbreviations to full names
# Note: for any teams that changed their names since the 2014 season, all games will refer to them using their current 
# name, even if they had a different name at the time of the game. 

def fix_team_names(game):
    team_mapping = {
        'ARI':'Arizona Cardinals',
        'ATL':'Atlanta Falcons',
        'BAL':'Baltimore Ravens',
        'BUF':'Buffalo Bills',
        'CAR':'Carolina Panthers',
        'CHI':'Chicago Bears',
        'CIN':'Cincinnati Bengals',
        'CLE':'Cleveland Browns',
        'DAL':'Dallas Cowboys',
        'DEN':'Denver Broncos',
        'DET':'Detroit Lions',
        'GB':'Green Bay Packers',
        'HOU':'Houston Texans',
        'IND':'Indianapolis Colts',
        'JAX':'Jacksonville Jaguars',
        'KC':'Kansas City Chiefs',
        'OAK':'Las Vegas Raiders',
        'LV':'Las Vegas Raiders',
        'LAC':'Los Angeles Chargers',
        'LAR':'Los Angeles Rams',
        'LA':'Los Angeles Rams',
        'MIA':'Miami Dolphins',
        'MIN':'Minnesota Vikings',
        'NE':'New England Patriots',
        'NO':'New Orleans Saints',
        'NYG':'New York Giants',
        'NYJ':'New York Jets',
        'PHI':'Philadelphia Eagles',
        'PIT':'Pittsburgh Steelers',
        'SF':'San Francisco 49ers',
        'SEA':'Seattle Seahawks',
        'TB':'Tampa Bay Buccaneers',
        'TEN':'Tennessee Titans',
        'WSH':'Washington Football Team',
        'WAS':'Washington Football Team'
    }
    
    return team_mapping[game['team']]
    
def fix_opponent_team_names(game):
    team_mapping = {
        'ARI':'Arizona Cardinals',
        'ATL':'Atlanta Falcons',
        'BAL':'Baltimore Ravens',
        'BUF':'Buffalo Bills',
        'CAR':'Carolina Panthers',
        'CHI':'Chicago Bears',
        'CIN':'Cincinnati Bengals',
        'CLE':'Cleveland Browns',
        'DAL':'Dallas Cowboys',
        'DEN':'Denver Broncos',
        'DET':'Detroit Lions',
        'GB':'Green Bay Packers',
        'HOU':'Houston Texans',
        'IND':'Indianapolis Colts',
        'JAX':'Jacksonville Jaguars',
        'KC':'Kansas City Chiefs',
        'OAK':'Las Vegas Raiders',
        'LV':'Las Vegas Raiders',
        'LAC':'Los Angeles Chargers',
        'LAR':'Los Angeles Rams',
        'LA':'Los Angeles Rams',
        'MIA':'Miami Dolphins',
        'MIN':'Minnesota Vikings',
        'NE':'New England Patriots',
        'NO':'New Orleans Saints',
        'NYG':'New York Giants',
        'NYJ':'New York Jets',
        'PHI':'Philadelphia Eagles',
        'PIT':'Pittsburgh Steelers',
        'SF':'San Francisco 49ers',
        'SEA':'Seattle Seahawks',
        'TB':'Tampa Bay Buccaneers',
        'TEN':'Tennessee Titans',
        'WSH':'Washington Football Team',
        'WAS':'Washington Football Team'
    }
    
    return team_mapping[game['opponent']]

nfl_per_game = nfl_per_game.reset_index()
nfl_per_game['team_full_name'] = nfl_per_game.apply(lambda x: fix_team_names(x), axis=1)
nfl_per_game['opponent_full_name'] = nfl_per_game.apply(lambda x: fix_opponent_team_names(x), axis=1)

In [90]:
# Add week column to elo data 

keep_cols = [col for col in nfl_elo.columns]
keep_cols.append('week')
nfl_elo = nfl_elo.merge(nfl_per_game, how='inner',
                                 left_on=['season', 'home_full_name', 'away_full_name', 'home_score', 'away_score'],
                                 right_on = ['season', 'team_full_name', 'opponent_full_name', 'score', 'opponent_score'], 
                                 suffixes = [None, '_'])
nfl_elo = nfl_elo[keep_cols]

# nfl_elo

,date,season,home,away,elo1_pre,elo2_pre,elo_prob1,elo_prob2,elo1_post,elo2_post,qbelo1_pre,qbelo2_pre,qb1,qb2,qb1_value_pre,qb2_value_pre,qb1_adj,qb2_adj,qbelo_prob1,qbelo_prob2,qb1_game_value,qb2_game_value,qb1_value_post,qb2_value_post,qbelo1_post,qbelo2_post,home_score,away_score,quality,importance,total_rating,home_full_name,away_full_name,week
0,2021-09-09,2021,TB,DAL,1656.235930,1452.424236,0.824539,0.175461,1659.671437,1448.988729,1634.451744,1505.628824,Tom Brady,Dak Prescott,213.410550,231.491799,8.527869,17.881196,0.736090,0.263910,317.128566,397.401725,223.782352,248.082791,1639.815956,1500.264612,31,29,85.0,48.0,67.0,Tampa Bay Buccaneers,Dallas Cowboys,1
1,2021-09-12,2021,IND,SEA,1561.490084,1565.751134,0.586531,0.413469,1530.547371,1596.693848,1527.521731,1553.102962,Carson Wentz,Russell Wilson,102.469223,187.030653,-21.756851,-0.326869,0.522245,0.477755,216.666972,246.652318,113.888998,192.992819,1500.541394,1580.083300,16,28,63.0,73.0,68.0,Indianapolis Colts,Seattle Seahawks,1
2,2021-09-12,2021,TEN,ARI,1553.397217,1486.436956,0.681272,0.318728,1506.171519,1533.662654,1534.793055,1483.219040,Ryan Tannehill,Kyler Murray,207.780411,217.216865,6.311825,5.662860,0.657171,0.342829,33.684565,302.017311,190.370826,225.696909,1489.651053,1528.361042,13,38,56.0,59.0,58.0,Tennessee Titans,Arizona Cardinals,1
3,2021-09-12,2021,DET,SF,1390.645249,1490.665951,0.449771,0.550229,1371.189983,1510.121217,1369.557634,1548.057844,Jared Goff,Jimmy Garoppolo,160.639094,127.748569,-1.519108,-4.082609,0.343427,0.656573,269.384920,175.216924,171.513676,132.495404,1355.200609,1562.414869,33,41,23.0,45.0,34.0,Detroit Lions,San Francisco 49ers,1
4,2021-09-12,2021,ATL,PHI,1484.493617,1426.465972,0.670004,0.329996,1437.713081,1473.246508,1462.273784,1432.000543,Matt Ryan,Jalen Hurts,178.596517,119.529623,0.995672,-7.458145,0.635255,0.364745,17.562049,382.369867,162.493070,145.813648,1418.481268,1475.793058,6,32,21.0,57.0,39.0,Atlanta Falcons,Philadelphia Eagles,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280,2022-01-23,2021,TB,LAR,1681.797158,1632.341105,0.659005,0.340995,1662.522916,1651.615346,1649.547292,1615.165080,Tom Brady,Matthew Stafford,249.882409,179.870436,11.456236,-0.623739,0.681440,0.318560,132.180809,379.027146,238.112249,199.786107,1629.446888,1635.265484,27,30,96.0,100.0,98.0,Tampa Bay Buccaneers,Los Angeles Rams,20
281,2022-01-23,2021,KC,BUF,1699.470145,1683.332367,0.614689,0.385311,1713.932369,1668.870143,1687.450699,1656.910634,Patrick Mahomes,Josh Allen,273.771767,275.630099,20.577067,18.202487,0.652720,0.347280,622.042187,434.101420,308.598809,291.477231,1700.324756,1644.036577,42,36,100.0,100.0,100.0,Kansas City Chiefs,Buffalo Bills,20
282,2022-01-30,2021,KC,CIN,1713.932369,1569.498961,0.769519,0.230481,1690.352000,1593.079331,1700.324756,1559.717895,Patrick Mahomes,Joe Burrow,308.598809,190.637925,36.961734,24.405309,0.810417,0.189583,134.683170,153.972640,291.207245,186.971396,1674.943733,1585.098917,24,27,98.0,100.0,99.0,Kansas City Chiefs,Cincinnati Bengals,21
283,2022-01-30,2021,LAR,SF,1651.615346,1621.465562,0.633608,0.366392,1661.352751,1611.728157,1635.265484,1631.632019,Matthew Stafford,Jimmy Garoppolo,199.786107,117.398901,9.365284,-11.464001,0.636093,0.363907,248.504596,197.148060,204.657956,125.373817,1644.929010,1621.968494,20,17,96.0,100.0,98.0,Los Angeles Rams,San Francisco 49ers,21


In [91]:
# Get important passing/qb related stats

def get_passing_stats(nfl, elo):
    passing = nfl[['season', 'week', 'team_full_name', 'total_pass_attempts', 'total_passing_yards', 'completions', 'passing_epa',
                  'pass_tds', 'avg_cpoe', 'qb_epa', 'sacks_allowed', 'interceptions_thrown']]
    home_qb_elos = elo[['season', 'week', 'home_full_name', 'qb1', 'qb1_value_pre', 'qb1_adj']]
    home_qb_elos.columns = ['season', 'week', 'team_full_name', 'qb', 'qb_value_pre', 'qb_adj']
    away_qb_elos = elo[['season', 'week', 'away_full_name', 'qb2', 'qb2_value_pre', 'qb2_adj']]
    away_qb_elos.columns = ['season', 'week', 'team_full_name', 'qb', 'qb_value_pre', 'qb_adj']
    qb_elos = pd.concat([home_qb_elos, away_qb_elos])
    passing = passing.merge(qb_elos, how='inner', 
                           left_on = ['season', 'week', 'team_full_name'], 
                           right_on = ['season', 'week', 'team_full_name'])
    
    prev_games_qb_rolling = passing[['total_pass_attempts', 'total_passing_yards', 'completions', 'passing_epa',
                                    'pass_tds', 'avg_cpoe', 'qb_epa', 'sacks_allowed', 'interceptions_thrown',
                                    'qb_value_pre', 'qb_adj', 'qb']].groupby(by=['qb']).rolling(6, min_periods=1, closed='left').mean()
    prev_games_qb_rolling.columns = prev_games_qb_rolling.columns + '_last_6_avg'
    prev_games_qb_rolling = passing[['season', 'week', 'team_full_name']].join(prev_games_qb_rolling.reset_index(level='qb'))
    
    return passing, prev_games_qb_rolling

In [92]:
# Get important rushing stats

def get_rushing_stats(nfl):
    rushing = nfl[['season', 'week', 'team_full_name', 'total_rushes', 'total_rush_yards', 'rushing_epa', 'rush_tds', 'fumbles', 
                  'lost_fumbles']]
    prev_games_rushing = rushing[['team_full_name', 'total_rushes', 'total_rush_yards', 'rushing_epa', 'rush_tds', 'fumbles', 
                  'lost_fumbles']]
    prev_games_rushing['ypc'] = prev_games_rushing.apply(lambda x: x.total_rush_yards/x.total_rushes, axis=1)
    rushing_rolling = prev_games_rushing.groupby(by=['team_full_name']).rolling(6, min_periods=1, closed='left').mean()
    rushing_rolling.columns = rushing_rolling.columns + '_last_6_avg'
    rushing_rolling = rushing[['season', 'week']].join(rushing_rolling.reset_index(level='team_full_name'))
    
    return rushing_rolling

In [93]:
# Get important defensive stats

def get_defense_and_st_stats(nfl):
    defense = nfl[['season', 'week', 'team_full_name', 'total_rushes_allowed', 'total_rush_yards_allowed', 'rush_tds_allowed',
                  'total_pass_attempts_allowed', 'passing_yards_allowed', 'completions_allowed', 
                  'pass_tds_allowed', 'avg_cpoe_allowed', 'sacks', 'fumbles_forced', 'interceptions', 'fumbles_recovered',
                  'special_teams_epa', 'passing_epa_def', 'rushing_epa_def']]
    prev_games_defense = defense[['team_full_name', 'total_rushes_allowed', 'total_rush_yards_allowed', 'rush_tds_allowed',
                  'total_pass_attempts_allowed', 'passing_yards_allowed', 'completions_allowed', 
                  'pass_tds_allowed', 'avg_cpoe_allowed', 'sacks', 'fumbles_forced', 'interceptions', 'fumbles_recovered',
                  'special_teams_epa', 'passing_epa_def', 'rushing_epa_def']]
    defense_rolling = prev_games_defense.groupby(by=['team_full_name']).rolling(6, min_periods=1, closed='left').mean()
    defense_rolling.columns = defense_rolling.columns + '_last_6_avg'
    defense_rolling = defense[['season', 'week']].join(defense_rolling.reset_index(level='team_full_name'))
    
    return defense_rolling